# GraphRAG Patterns with Neo4j

This notebook demonstrates how to use Neo4j's GraphRAG library to implement retrieval-augmented generation with graph context.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from graphrag import GraphRAG

# Load environment variables
load_dotenv()

# Initialize Neo4j connection
URI = os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize GraphRAG
graph_rag = GraphRAG(
    uri=URI,
    username=AUTH[0],
    password=AUTH[1],
    openai_api_key=OPENAI_API_KEY
)

## Create Vector Index

First, let's create a vector index for our documents:

In [ ]:
# Create vector index on Document nodes
graph_rag.create_vector_index(
    index_name="document_embeddings",
    node_label="Document",
    property_name="content"
)

## Pattern 1: Document Question-Answering

Let's ask questions about our product documentation:

In [ ]:
# Ask a question about laptop troubleshooting
question = "What are the steps to troubleshoot laptop connectivity issues?"

answer = graph_rag.ask(
    question=question,
    cypher_query="""
    MATCH (d:Document)
    WHERE d.type = 'manual' OR d.type = 'support_case'
    RETURN d.content as text
    """
)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

## Pattern 2: Context-Enhanced Responses

Now let's use graph relationships to enhance our responses:

In [ ]:
# Ask about a specific product with context
question = "What are common issues with the Laptop Pro model?"

answer = graph_rag.ask(
    question=question,
    cypher_query="""
    // Find product documentation
    MATCH (p:Product {name: 'Laptop Pro'})-[:HAS_MANUAL]->(m:Document)
    
    // Find related support cases
    OPTIONAL MATCH (p)<-[:ABOUT]-(c:Document)
    WHERE c.type = 'support_case'
    
    // Return both manual content and support cases
    RETURN 
        m.content as manual,
        collect(c.content) as support_cases
    """
)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

## Pattern 3: Multi-Hop Knowledge Retrieval

Let's use graph traversal to gather related information:

In [ ]:
# Ask about related product issues
question = "What issues are common across laptop models?"

answer = graph_rag.ask(
    question=question,
    cypher_query="""
    // Find products in the laptop category
    MATCH (cat:Category {name: 'Laptops'})<-[:IN_CATEGORY]-(p:Product)
    
    // Get support cases for these products
    MATCH (p)<-[:ABOUT]-(c:Document)
    WHERE c.type = 'support_case'
    
    // Return product info and support cases
    RETURN 
        p.name as product,
        collect(c.content) as cases
    """
)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

## Pattern 4: Customer Support Context

Let's use customer history to provide personalized responses:

In [ ]:
# Ask about customer-specific issues
customer_id = "C1"
question = "What issues has this customer experienced with their products?"

answer = graph_rag.ask(
    question=question,
    cypher_query="""
    // Find customer's products
    MATCH (c:Customer {id: $customer_id})-[:PLACED_ORDER]->(o:Order)-[:CONTAINS]->(p:Product)
    
    // Get support cases for these products
    MATCH (c)-[:HAS_CASE]->(sc:Document)
    WHERE sc.type = 'support_case'
    
    // Get product manuals
    MATCH (p)-[:HAS_MANUAL]->(m:Document)
    
    // Return all context
    RETURN 
        c.name as customer,
        collect(DISTINCT p.name) as products,
        collect(DISTINCT sc.content) as support_cases,
        collect(DISTINCT m.content) as manuals
    """,
    params={"customer_id": customer_id}
)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

## Key GraphRAG Features Used

1. **Vector Search**: Automatic embedding and similarity search
2. **Graph Context**: Using graph relationships to enhance retrieval
3. **Cypher Integration**: Flexible graph queries for context gathering
4. **Multi-Document RAG**: Combining information from multiple sources
5. **Parameter Support**: Dynamic querying based on input parameters